# Assigment 2 - Game 0: Connect-4

## Instructions

This is a **self-contained notebook** - everything you need is here!

### Quick Start
1. **Update configuration** in Section 1 below
2. **Run all cells** up to Section 4 (this loads the game client)
3. **Implement your solver** in Section 5
4. **Play the game** in Section 7

### What You Need To Do
- Focus ONLY on implementing `my_agent()` function (Section 5)
  - You can create various players, you'll be able to select your prefered one.
- Everything else is provided for you!


---
## Section 1: Identifying token

**⚠️ UPDATE THIS VALUE**

In [18]:
STUDENT_TOKEN = 'YOUR-NAME'  # e.g., 'JOHN-DOE'

---
## Section 2: Setup

**Run this cell (no changes needed)**

In [19]:
import requests
import json
import time
import random
from typing import List, Optional, Tuple, Any, Dict
from copy import deepcopy

print("✅ Dependencies imported")

BASE_URL = 'https://ie-aireasoning-gr4r5bl6tq-ew.a.run.app'

print("✅ Configuration loaded")

✅ Dependencies imported
✅ Configuration loaded


---
## Section 3: Game Client Library

**Run this cell (no changes needed)**

This defines the game client that handles all server communication.

In [20]:
class GameClient:
    def __init__(self, base_url: str, token: str, debug: bool = False):
        self.base_url = base_url.rstrip('/')
        self.token = token
        self.debug = debug

    def _make_request(self, endpoint: str, params: dict, max_retries: int = 10) -> dict:
        params['TOKEN'] = self.token
        url = f'{self.base_url}{endpoint}'

        for attempt in range(max_retries):
            try:
                if self.debug:
                    print(f"[DEBUG] Request: {endpoint}")
                    print(f"[DEBUG] Params: {params}")

                response = requests.get(url, params=params, timeout=30)

                if self.debug:
                    print(f"[DEBUG] Response [{response.status_code}]: {response.text[:200]}")

                if response.status_code == 200:
                    if response.text:
                        try:
                            return response.json()
                        except (json.JSONDecodeError, ValueError) as e:
                            if self.debug:
                                print(f"[DEBUG] Non-JSON response: {response.text[:100]}")
                            return {}
                    return {}
                else:
                    print(f"⚠️  HTTP {response.status_code}: {response.text[:200]}")

            except requests.exceptions.Timeout:
                print(f"⚠️  Request timeout (attempt {attempt + 1}/{max_retries})")
            except requests.exceptions.RequestException as e:
                print(f"⚠️  Request error: {e} (attempt {attempt + 1}/{max_retries})")
            except Exception as e:
                print(f"⚠️  Unexpected error: {type(e).__name__}: {e} (attempt {attempt + 1}/{max_retries})")

            if attempt < max_retries - 1:
                time.sleep(1)

        raise Exception(f"Failed to connect to {endpoint} after {max_retries} attempts")

    def create_match(self, game_type: str, num_games: int, multiplayer: bool = False) -> str:
        response = self._make_request('/new-match', {
            'game-type': game_type,
            'num-games': str(num_games),
            'multi-player': 'True' if multiplayer else 'False'
        })
        
        if 'match-id' not in response:
            print(f"❌ Server response missing 'match-id'. Response: {response}")
            raise KeyError(f"Server response missing 'match-id'. Got: {response}")
        
        return response['match-id']

    def join_match(self, match_id: str) -> dict:
        response = self._make_request('/join-match', {
            'match-id': match_id
        })
        return response

    def get_game_state(self, match_id: str, game_index: int) -> dict:
        return self._make_request('/game-state-in-match', {
            'match-id': match_id,
            'game-index': str(game_index)
        })

    def get_match_state(self, match_id: str) -> dict:
        return self._make_request('/match-state', {
            'match-id': match_id
        })

    def make_move(self, match_id: str, player: str, move: Any) -> bool:
        move_str = move if isinstance(move, str) else json.dumps(move)
        
        self._make_request('/make-move-in-match', {
            'match-id': match_id,
            'player': player,
            'move': move_str
        })
        return True

print("✅ GameClient loaded")

✅ GameClient loaded


---
## Section 4: Game State Class & Helper Functions

**Run this cell (no changes needed)**

This defines the `ConnectFourGame` class with all helper methods you'll need.

In [21]:
class ConnectFourGame:
    """
    Represents Connect Four game state with helper methods.
    
    Key methods for your solver:
    - game.board                        # 2D list of board
    - game.get_valid_moves()           # List of valid columns
    - game.simulate_move(col)          # Simulate move for search
    - game.check_winner_on_board(board, player)  # Check winner
    - game.is_board_full(board)        # Check for draw
    - game.print_board()               # Debug visualization
    """

    def __init__(self, state: str, status: str, current_player: str):
        self.state = state
        self.status = status
        self.current_player = current_player
        self._board = None
        self._valid_moves = None

    def update(self, state: str, status: str, current_player: str):
        """Update game state with new information from server."""
        self.state = state
        self.status = status
        self.current_player = current_player
        self._board = None  # Clear cached board
        self._valid_moves = None  # Clear cached moves

    @property
    def board(self) -> List[List[str]]:
        """Get board as 2D list. board[row][col] where row 0 is top."""
        if self._board is None:
            self._board = json.loads(self.state)
        return self._board

    def get_valid_moves(self) -> List[int]:
        """Get list of valid column indices."""
        if self._valid_moves is None:
            board = self.board
            self._valid_moves = [
                col for col in range(len(board[0]))
                if board[0][col] == '.'
            ]
        return self._valid_moves

    def is_terminal(self) -> bool:
        """Check if game is over."""
        return self.status == 'complete'

    def is_waiting(self) -> bool:
        """Check if waiting for opponent."""
        return self.status == 'waiting'

    def get_winner(self) -> Optional[str]:
        """Get winner ('X', 'O', '-' for draw, None if ongoing)."""
        if not self.is_terminal():
            return None
        return self.current_player

    def get_opponent(self, player: str) -> str:
        """Get opponent's symbol."""
        return 'O' if player == 'X' else 'X'

    def simulate_move(self, column: int, player: Optional[str] = None) -> List[List[str]]:
        """
        Simulate placing a piece in a column.
        Returns new board state (does NOT modify original or contact server).
        Essential for minimax/alpha-beta algorithms!
        """
        if player is None:
            player = self.current_player

        new_board = [row[:] for row in self.board]
        
        for row in range(len(new_board) - 1, -1, -1):
            if new_board[row][column] == '.':
                new_board[row][column] = player
                break

        return new_board

    def check_winner_on_board(self, board: List[List[str]], player: str) -> bool:
        """
        Check if player has won on given board.
        Use this to evaluate terminal states in your search.
        """
        rows = len(board)
        cols = len(board[0])
        
        for row in range(rows):
            for col in range(cols - 3):
                if all(board[row][col + i] == player for i in range(4)):
                    return True

        for row in range(rows - 3):
            for col in range(cols):
                if all(board[row + i][col] == player for i in range(4)):
                    return True

        for row in range(rows - 3):
            for col in range(cols - 3):
                if all(board[row + i][col + i] == player for i in range(4)):
                    return True

        for row in range(rows - 3):
            for col in range(3, cols):
                if all(board[row + i][col - i] == player for i in range(4)):
                    return True

        return False

    def is_board_full(self, board: Optional[List[List[str]]] = None) -> bool:
        """Check if board is full (draw condition)."""
        if board is None:
            board = self.board
        return all(cell != '.' for cell in board[0])

    def print_board(self):
        """Print nice board representation."""
        board = self.board
        cols = len(board[0])

        print("\n" + "=" * (cols * 2 + 1))
        for row in board:
            print("|" + "|".join(row) + "|")
        print("=" * (cols * 2 + 1))
        print(" " + " ".join(str(i) for i in range(cols)))
        print()


def play_game(
    solver,
    base_url: str,
    token: str,
    game_type: str = 'connect4',
    multiplayer: bool = False,
    match_id: Optional[str] = None,
    num_games: int = 1,
    debug: bool = False,
    verbose: bool = True
) -> Tuple:
    client = GameClient(base_url, token, debug=debug)

    if match_id is None:
        if verbose:
            print(f"🎮 Creating new match: {num_games} x {game_type}")
        match_id = client.create_match(game_type, num_games, multiplayer)
        if verbose:
            print(f"   Match ID: {match_id}")

    if verbose:
        print(f"🔗 Joining match {match_id}...")
    match = client.join_match(match_id)
    player = match['player']
    num_games = match.get('num-games', num_games)
    if verbose:
        print(f"   You are player: {player}")

    game_state = client.get_game_state(match_id, 0)
    if game_state['status'] == 'waiting':
        if verbose:
            print("⏳ Waiting for opponent to join...")
        while game_state['status'] == 'waiting':
            time.sleep(2)
            game_state = client.get_game_state(match_id, 0)

    all_results = []
    wins = 0
    losses = 0
    draws = 0

    while True:
        match_state = client.get_match_state(match_id)
        if match_state['status'] != 'in_progress':
            break
        game_num = match_state['current-game-index']

        if verbose:
            print(f"\n{'='*50}")
            print(f"🎮 GAME {game_num + 1}/{num_games}")
            print(f"{'='*50}\n")

        # Get initial game state and check player assignment
        game_state = client.get_game_state(match_id, game_num)
        
        # Update player sign if it changed (randomized per game)
        if 'my-player' in game_state and game_state['my-player']:
            new_player = game_state['my-player']
            if new_player != player and verbose and game_num > 0:
                print(f"ℹ️  Player assignment changed: You are now Player {new_player}\n")
            player = new_player
        
        # Create game object ONCE per game
        game = ConnectFourGame(game_state['state'], game_state['status'], game_state['player'])

        move_count = 0
        while game_state['status'] != 'complete':
            game_state = client.get_game_state(match_id, game_num)
            player = game_state['my-player']
            if 'winner' in game_state:
                break

            game.update(game_state['state'], game_state['status'], game_state['player'])
            
            if game.is_terminal():
                if verbose:
                    print("Final board: ")
                    game.print_board()
                break

            if verbose:
                game.print_board()

            if game.current_player == player:
                if verbose:
                    print(f"🤔 Your turn (Player {player})...")

                try:
                    move = solver(game)

                    if move not in game.get_valid_moves():
                        print(f"❌ Invalid move {move}! Valid moves: {game.get_valid_moves()}")
                    else:
                        if verbose:
                            print(f"   Playing column {move}")

                        client.make_move(match_id, player, move)
                    move_count += 1

                except Exception as e:
                    print(f"❌ Error in solver: {e}")
                    import traceback
                    traceback.print_exc()
                    if num_games == 1:
                        return 'error', None
                    else:
                        all_results.append(('error', None))
                        break
            else:
                if verbose:
                    print(f"⏳ Waiting for opponent (Player {game.current_player})...")
                time.sleep(2)

        # Update game one final time with terminal state
        game.update(game_state['state'], game_state['status'], game_state.get('player', game.current_player))
        
        if verbose:
            game.print_board()
            print("=" * 40)

        winner = game_state['winner']
        if winner == '-':
            if verbose:
                print("🤝 Game ended in a DRAW!")
            result = 'draw'
            draws += 1
        elif winner == player:
            if verbose:
                print("🎉 You WON! Congratulations!")
            result = 'win'
            wins += 1
        else:
            if verbose:
                print("😞 You LOST. Better luck next time!")
            result = 'loss'
            losses += 1

        all_results.append((result, winner))

        if verbose and num_games > 1:
            print(f"\n📊 Current Record: {wins}W - {losses}L - {draws}D")
            print(f"   Games Remaining: {num_games - game_num - 1}\n")

    # Return results
    stats = {
        'wins': wins,
        'losses': losses,
        'draws': draws,
        'total_games': num_games,
        'win_rate': wins / num_games if num_games > 0 else 0,
        'player': player,
        'match_id': match_id
    }
    return stats, all_results

print("✅ Game library loaded")
print("✅ Ready to implement your solver!")

✅ Game library loaded
✅ Ready to implement your solver!


---
## Section 5: YOUR SOLVER IMPLEMENTATION

**⭐ THIS IS WHERE YOU WRITE YOUR CODE! ⭐**

Implement your AI algorithm here. You can use:
- Minimax
- Alpha-beta pruning
- Custom heuristics

### Available Methods

```python
game.board                          # 2D list of board
game.get_valid_moves()             # List of valid column indices [0, 1, 3, ...]
game.current_player                # Your player symbol ('X' or 'O')
game.get_opponent(player)          # Get opponent symbol
game.simulate_move(col, player)    # Simulate move, returns new board
game.check_winner_on_board(board, player)  # Check if player won
game.is_board_full(board)          # Check if board is full (draw)
game.print_board()                 # Print board for debugging
```

In [22]:
def my_agent(game: ConnectFourGame) -> int:
    """
    Your AI implementation.
    
    Args:
        game: ConnectFourGame object with helper methods
    
    Returns:
        int: Column index to play (0-based)
    """
    
    # Get basic info
    player = game.current_player
    opponent = game.get_opponent(player)
    valid_moves = game.get_valid_moves()
    board = game.board
    
    # ============================================================
    # TODO: IMPLEMENT YOUR ALGORITHM HERE!
    # ============================================================
    
    # Example: Random move (replace with your algorithm)
    return random.choice(valid_moves)
    
    # ============================================================
    # Some ideas to implement:
    # 1. Minimax algorithm with depth limit
    # 2. Alpha-beta pruning for better performance
    # 3. Evaluation function for non-terminal states
    # 4. Iterative deepening
    #
    # Make sure to use helper functions to make the code readable
    # ============================================================

print("✅ Solver function defined")
print("   Remember to implement your algorithm before running!")

✅ Solver function defined
   Remember to implement your algorithm before running!


---
## Section 6: Test Your Solver (Optional)

Test parts of your implementation before playing a full game.

In [23]:
# Create a test board
test_board = [
    ['.', '.', '.', '.'],
    ['.', '.', '.', '.'],
    ['.', 'X', '.', '.'],
    ['X', 'O', 'O', '.']
]

print("Test board:")
for row in test_board:
    print(' '.join(row))

# Create a mock game object for testing
test_game_state = json.dumps(test_board)
test_game = ConnectFourGame(test_game_state, 'playing', 'X')

print(f"\nValid moves: {test_game.get_valid_moves()}")
print(f"Board full: {test_game.is_board_full()}")

# Test simulate_move
print("\nSimulating move in column 2 for X:")
new_board = test_game.simulate_move(2, 'X')
for row in new_board:
    print(' '.join(row))

Test board:
. . . .
. . . .
. X . .
X O O .

Valid moves: [0, 1, 2, 3]
Board full: False

Simulating move in column 2 for X:
. . . .
. . . .
. X X .
X O O .


---
## Section 7: Manual Play Mode (Try the Game Yourself!)

**Play Connect Four manually** to understand the game before implementing your AI!

This lets you:
- Experience the game firsthand
- Test the server connection
- Understand winning strategies
- Play against the server AI

In [24]:
def manual_player_solver(game: ConnectFourGame) -> int:
    """
    Interactive manual player - YOU choose the moves!
    Perfect for testing the game and understanding the rules.
    """
    game.print_board()
    valid_moves = game.get_valid_moves()
    
    print(f"\n🎮 YOUR TURN (Player {game.current_player})!")
    print(f"Valid columns: {valid_moves}")
    
    while True:
        try:
            move = input("Enter column number (or 'q' to quit): ").strip()
            
            if move.lower() == 'q':
                print("Quitting game...")
                raise KeyboardInterrupt()
            
            move = int(move)
            
            if move in valid_moves:
                return move
            else:
                print(f"❌ Invalid! Column {move} is full or out of range.")
                print(f"   Valid columns: {valid_moves}")
        except ValueError:
            print("❌ Please enter a number or 'q' to quit.")
        except KeyboardInterrupt:
            print("\n👋 Thanks for playing!")
            raise


print("✅ Manual player loaded")
print("   Run the cell below to play interactively!")

✅ Manual player loaded
   Run the cell below to play interactively!


---
## Section 8: Play the Game!

**Run this cell to test your solver against the AI**

In [25]:
SOLVER = my_agent
GAME_TYPE = 'connect4'
MULTIPLAYER = False
MATCH_ID = None
NUM_GAMES = 1

result = play_game(
    solver=SOLVER,
    base_url=BASE_URL,
    token=STUDENT_TOKEN,
    game_type=GAME_TYPE,
    multiplayer=MULTIPLAYER,
    match_id=MATCH_ID,
    num_games=NUM_GAMES,
    debug=False,
    verbose=True
)

stats, all_results = result
print("\n📊 Summary:")
print(f"   Record: {stats['wins']}W - {stats['losses']}L - {stats['draws']}D")
print(f"   Win Rate: {stats['win_rate']*100:.1f}%")

🎮 Creating new match: 1 x connect4
   Match ID: 294
🔗 Joining match 294...
   You are player: X

🎮 GAME 1/1


|.|.|.|.|
|.|.|.|.|
|.|.|.|.|
|.|.|.|.|
 0 1 2 3

🤔 Your turn (Player X)...
   Playing column 0

|.|.|.|.|
|.|.|.|.|
|.|.|.|.|
|X|O|.|.|
 0 1 2 3

🤔 Your turn (Player X)...
   Playing column 3

|.|.|.|.|
|.|.|.|.|
|.|O|.|.|
|X|O|.|X|
 0 1 2 3

🤔 Your turn (Player X)...
   Playing column 1

|.|.|.|.|
|.|X|.|.|
|.|O|.|.|
|X|O|O|X|
 0 1 2 3

🤔 Your turn (Player X)...
   Playing column 0

|.|.|.|.|
|.|X|.|.|
|X|O|.|O|
|X|O|O|X|
 0 1 2 3

🤔 Your turn (Player X)...
   Playing column 1

|.|X|.|.|
|.|X|.|O|
|X|O|.|O|
|X|O|O|X|
 0 1 2 3

🤔 Your turn (Player X)...
   Playing column 2

|.|X|.|.|
|.|X|O|O|
|X|O|X|O|
|X|O|O|X|
 0 1 2 3

🤔 Your turn (Player X)...
   Playing column 0

|.|X|.|O|
|X|X|O|O|
|X|O|X|O|
|X|O|O|X|
 0 1 2 3

🤔 Your turn (Player X)...
   Playing column 0

|X|X|.|O|
|X|X|O|O|
|X|O|X|O|
|X|O|O|X|
 0 1 2 3

🎉 You WON! Congratulations!

📊 Summary:
   Record: 1W - 0L - 0D
